In [7]:
pip install -U liblinear-official

In [8]:
import urllib.request
import numpy as np
import math
from sklearn.preprocessing import PolynomialFeatures
from liblinear.liblinearutil import *

In [9]:
def getData(url):
  #get data 
  contents = urllib.request.urlopen(url).read()
  text = str(contents,'utf-8')
  # data preprocessing
  sptext = text.split('\n')

  X = []
  Y = []

  for i in range(len(sptext)-1):
    data = sptext[i].split(' ')
    # print(data)
    label = data[-1]
    # print(type(data))
    data.pop()
    # print(len(data))
    X += [data]
    Y += [label]

  X = np.asarray(X, dtype = float)
  Y = np.asarray(Y, dtype = float)
  return X, Y

if __name__ == '__main__':
  # Get Data
  train_data = "https://www.csie.ntu.edu.tw/~htlin/course/ml21fall/hw4/hw4_train.dat"
  train_X ,train_Y = getData(train_data)
  
  # Third-order Polynomial Transformation
  poly = PolynomialFeatures(3)
  train_X = poly.fit_transform(train_X)
  
  # 5-fold Cross Validation
  cv_train_X = []
  cv_train_Y = []
  cv_val_X = []
  cv_val_Y = []
  for i in range(0, len(train_X), 40):
    if i == 0:
      cv_train_X.append(train_X[i+40:])
      cv_train_Y.append(train_Y[i+40:])
    else:
      cv_train_X.append(np.concatenate((train_X[:i], train_X[i+40:])))
      cv_train_Y.append(np.concatenate((train_Y[:i], train_Y[i+40:])))

  for i in range(0, len(train_X), 40):
    cv_val_X.append(train_X[i:i+40])
    cv_val_Y.append(train_Y[i:i+40])

  # C = 1/(2*lambda)
  lamb = [0.0001, 0.01, 1, 100, 10000]
  C = []
  for l in lamb:
    C.append(str(1/(2*l)))

  # LIBLINEAR train and predict
  for c in C:
    print("c: ",c)
    AccvalN = 0
    for i in range(5):
      prob = problem(cv_train_Y[i], cv_train_X[i])
      param = parameter('-s 0 -c {} -e 0.000001'.format(c))
      m = train(prob, param)
      p_label, p_acc, p_val = predict(cv_val_Y[i], cv_val_X[i], m)
      # print(p_label)
      # print(p_acc)
      # print(p_val)
      AccvalN += p_acc[0]
    Ecv = (100 - (AccvalN/5))/ 100
    print("Ecv: ",Ecv)

c:  5000.0
Accuracy = 82.5% (33/40) (classification)
Accuracy = 82.5% (33/40) (classification)
Accuracy = 70% (28/40) (classification)
Accuracy = 82.5% (33/40) (classification)
Accuracy = 87.5% (35/40) (classification)
Ecv:  0.19
c:  50.0
Accuracy = 82.5% (33/40) (classification)
Accuracy = 82.5% (33/40) (classification)
Accuracy = 75% (30/40) (classification)
Accuracy = 85% (34/40) (classification)
Accuracy = 90% (36/40) (classification)
Ecv:  0.17
c:  0.5
Accuracy = 85% (34/40) (classification)
Accuracy = 87.5% (35/40) (classification)
Accuracy = 85% (34/40) (classification)
Accuracy = 92.5% (37/40) (classification)
Accuracy = 92.5% (37/40) (classification)
Ecv:  0.115
c:  0.005
Accuracy = 90% (36/40) (classification)
Accuracy = 90% (36/40) (classification)
Accuracy = 87.5% (35/40) (classification)
Accuracy = 97.5% (39/40) (classification)
Accuracy = 95% (38/40) (classification)
Ecv:  0.08
c:  5e-05
Accuracy = 72.5% (29/40) (classification)
Accuracy = 80% (32/40) (classification)
Acc